# Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

Today we will score documents with respect to user query using language models and also get some experience with topic modelling.

## Loading data

In this class we will use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads).

In [1]:
# TODO: read the dataset
def read_dataset(file_path):
    docs = []
    with open(file_path) as fp:
        for cnt, line in enumerate(fp):
            docs.append(line)
    return docs

fuel_data = read_dataset("topic_modeling_dataset/testdata_news_fuel_845docs.txt")
brain_inj_data = read_dataset("topic_modeling_dataset/testdata_braininjury_10000docs.txt")
economy_data = read_dataset("topic_modeling_dataset/testdata_news_economy_2073docs.txt")
music_data = read_dataset("topic_modeling_dataset/testdata_news_music_2084docs.txt")

all_data = fuel_data + brain_inj_data + economy_data + music_data

In [2]:
print("# of documents", len(all_data))
assert len(all_data) == 15002

# of documents 15002


In [3]:
all_data[0][:300] + "..."

'the new york times said editorial for tuesday jan new year day has way stealing down upon coming the shadow the christmas week and its own scintillating eve you celebrated midnight last night was easy little illusory think the festivities gesture dismissal warding off the old year and its associated...'

In [4]:
data_ranges = [("fuel", len(fuel_data)), ("brain_injury", len(fuel_data)+len(brain_inj_data)),
               ("economy", len(fuel_data)+len(brain_inj_data)+len(economy_data)), ("music", len(all_data))]

def get_doc_category(doc_id, ranges):
    for r in ranges:
        if doc_id < r[1]:
            return r[0]
    return "out of range"

print("data_ranges", data_ranges)

data_ranges [('fuel', 845), ('brain_injury', 10845), ('economy', 12918), ('music', 15002)]


## 1. Ranking Using Language Models
Our goal is to rank documents by *P(d|q)*, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: *P(d|q) = P(q|d)P(d)/P(q)*

*P(q)* is the same for all documents, and so can be ignored. The prior probability of a document *P(d)* is often treated as uniform across all *d* and so it can also be ignored. What does it mean? 

It means that comparing *P(q|d)* between different documents we can compare how relevant are they to the query. How can we estimate *P(q|d)*?

*P(q|d)* can be estimated as:
![](https://i.imgur.com/BEIMAC1.png)

where M<sub>d</sub> is the language model of document *d*, tf<sub>t,d</sub> is the term frequency of term *t* in document *d*, and L<sub>d</sub> is the number of tokens in document *d*. That is, we just count up how often each word occurred, and divide by the total number of words in the document *d*. The first thing we need to do is to build a term-document matrix for tour dataset.

In [5]:
# TODO: Build term-document matrix for the dataset
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
counts_data = vectorizer.fit_transform(all_data)
terms = vectorizer.get_feature_names()
print("vocabulary size", len(terms))
print("first 100 terms: ", terms[0:100])

vocabulary size 92067
first 100 terms:  ['000l', '000m', '000th', '03rd', '0as', '0dpt', '0gy', '0mm', '0yrs', '1000m', '1000w', '100ab', '100b', '100bb', '100beta', '100g', '100percent', '100th', '101st', '1027g', '103rd', '1040a', '1040s', '104th', '105th', '106s', '106th', '107th', '108th', '109th', '10am', '10fr', '10g', '10million', '10mm', '10mo', '10mwt', '10mwts', '10p10', '10s', '10th', '10x', '10y', '10years', '110million', '110mm', '110th', '112th', '113th', '114th', '115th', '116th', '117th', '118b', '118th', '1199s', '11a', '11b', '11c', '11million', '11th', '120min', '120th', '1223willsmith', '123i', '1240ut', '125cc', '125d', '125i', '125ml', '125ms', '125s', '125th', '126th', '127th', '128th', '1296c', '129th', '12h', '12m', '12million', '12mo', '12p2', '12p6', '12p70', '12th', '12u', '12v2', '1300s', '130f', '130j', '130million', '130r', '130s', '131xe', '135th', '136million', '137th', '138th', '139th']


### Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *additive smoothing* - adding a small number (0 to 1) to the observed counts and renormalizing to give a probability distribution.

Another option is called Jelinek-Mercer smoothing - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and M<sub>c</sub> is a language model built from the entire document collection.

Refer to *Chapter 12* for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes as an input tdm matrix, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [6]:
import numpy as np

def lm_rank_documents(query, tdm, terms_list, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    """
    :param query: dict, term:count            
    :param tdm: term-document matrix
    :param terms_list: vocabulary list
    :param smoothing: which smoothing to apply, either 'additive' or 'jelinek-mercer'
    :param param: alpha for additive / lambda for jelinek-mercer
    :return: list of scores, list of doc_ids sorted by their scores 
    """
    n_docs = tdm.shape[0]
    doc_lengths = tdm.sum(axis=1)
    len_collection = np.sum(doc_lengths)
    scores = np.zeros(n_docs)
    for term in query.keys():
        # check if term exists
        if term in terms_list:
            # get term's id
            term_id = terms_list.index(term)
        else:
            continue
        query_tf = query[term]
        # calculate collection frequency of a term
        collection_tf = np.sum(tdm[:, term_id])
        for doc_id in range(n_docs):
            doc_tf = tdm[doc_id, term_id]
            # apply smoothing of any
            if smoothing == 'additive':
                doc_score_factor = (doc_tf + param) / (doc_lengths[doc_id] + param*len(terms_list))
            elif smoothing == 'jelinek':
                doc_score_factor = param*doc_tf/doc_lengths[doc_id] + (1-param)*collection_tf/len_collection
            else:
                doc_score_factor = doc_tf/doc_lengths[doc_id]
            doc_score_factor = doc_score_factor**query_tf

            if doc_id not in scores:
                scores[doc_id] = 1
            # accumulate scores
            scores[doc_id] *= doc_score_factor
    # sort doc_ids by scores
    sorted_doc_ids = np.argsort(-scores)
    return scores, sorted_doc_ids

### Testing

Check if this type of ranking gives meaningful results. For each query output document category, doc_id, score, and the beginning of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [7]:
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
import string

def prepare_query(raw_query):
    # lower-case, remove punctuation and stopwords
    stop_words = list(string.punctuation) + stopwords.words('english')
    return Counter([i for i in word_tokenize(raw_query.lower()) if i not in stop_words])


def process_query(raw_query, counts_data, terms, data_ranges):
    # TODO process user query and print search results including document category, id, score, and some part of it
    query = prepare_query(raw_query)
    print("user query:", '\033[95m' + raw_query + '\033[0m')
    print("prepared query:", query)
    doc_scores, doc_ids_sorted = lm_rank_documents(query, counts_data, terms)
    print("\nsearch results:")
    for i in range(5):
        doc_id = doc_ids_sorted[i]
        print('\033[1m' + get_doc_category(doc_id, data_ranges) + '\033[0m', doc_id, doc_scores[doc_id])
        print(all_data[doc_id][:100] + "...")
    

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    process_query(q, counts_data, terms, data_ranges)
    print("\n")

user query: piano concert
prepared query: Counter({'piano': 1, 'concert': 1})

search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...


user query: symptoms of head trauma
prepared query: Counter({'symptoms': 1, 'head': 1, 'trauma': 1})

search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed

## 2. Topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [8]:
# TODO: apply LDA to our dataset and output the resulting categories 
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))


# Tweak the two parameters below
number_topics = 4
number_words = 20
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(counts_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, vectorizer, number_words)

Topics found via LDA:

Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
